In [1187]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sb
import sklearn
import statsmodels.api as sm
from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import metrics
from sklearn import preprocessing
#from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [1188]:
ds1 = pd.read_table("./data/DataSet1__24Apr2018.txt", header=0, sep=' ')
ds2 = pd.read_table("./data/DataSet2_24Apr2018.txt", header=0, sep=' ')
ds3 = pd.read_table("./data/DataSet3__24Apr2018.txt", header=0, sep=' ')
ds4 = pd.read_table("./data/DataSet4__24Apr2018.txt", header=0, sep=' ')

In [1189]:
ds1.columns.tolist()

['CommunityCode',
 'CommunityZipCode',
 'ProspectID',
 'AgeatInquiry',
 'CareLevel',
 'Gender',
 'Marital',
 'Veteran',
 'CurrentResidence',
 'ZipCode',
 'UnitPreference',
 'ProvidedEmail',
 'ProvidedPhone',
 'Budget',
 'RequestedVisit',
 'NoCall',
 'NoEmail',
 'InquiryDate',
 'ReferralCompany',
 'ProjectedMoveIn',
 'DeactDate',
 'DeactReason',
 'NotQual',
 'CurrentStatus']

In [1190]:
ds3.columns.tolist()

['ProspectID', 'ResidentID', 'dtmovein', 'dtmoveout', 'istatus', 'ireason']

In [1191]:
ds13 = pd.merge(ds1, ds3, on='ProspectID', how='left')

In [1192]:
ds13.columns.tolist()

['CommunityCode',
 'CommunityZipCode',
 'ProspectID',
 'AgeatInquiry',
 'CareLevel',
 'Gender',
 'Marital',
 'Veteran',
 'CurrentResidence',
 'ZipCode',
 'UnitPreference',
 'ProvidedEmail',
 'ProvidedPhone',
 'Budget',
 'RequestedVisit',
 'NoCall',
 'NoEmail',
 'InquiryDate',
 'ReferralCompany',
 'ProjectedMoveIn',
 'DeactDate',
 'DeactReason',
 'NotQual',
 'CurrentStatus',
 'ResidentID',
 'dtmovein',
 'dtmoveout',
 'istatus',
 'ireason']

In [1193]:
ds13.head()

,CommunityCode,CommunityZipCode,ProspectID,AgeatInquiry,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,...,ProjectedMoveIn,DeactDate,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason
0,731.0,54311.0,0f0d9652-7697-4d03-84da-000137b878db,NaN,A,Female,NaN,False,NaN,NaN,...,NaN,2012-09-18 05:00:00,Request Removal,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN
1,415.0,60010.0,59a04927-19ee-42de-840f-545a4e8b5533,77.0,I,Female,Widow,False,Living at Home,NaN,...,NaN,2013-01-19 06:00:00,NaN,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN
2,491.0,30052.0,87305eb0-2ed2-430c-9b4e-00019a38134a,NaN,NaN,Female,NaN,False,NaN,NaN,...,NaN,2014-12-17 05:00:00,Duplicate Lead,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN
3,775.0,33462.0,50dd6777-4333-48bb-9ac5-00041d493ef0,NaN,NaN,Female,NaN,False,NaN,NaN,...,NaN,2013-10-17 04:00:00,Duplicate Lead,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN
4,492.0,33021.0,f7ce16ab-bbaf-45ae-be5b-0004e020269f,NaN,A,Female,NaN,False,NaN,NaN,...,NaN,2014-12-24 05:00:00,Not Qualified,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN


In [1194]:
id_column = ['ProspectID']

In [1195]:
target_column = ['converted']

In [1196]:
boolean_columns = ['Veteran', 'ProvidedEmail', 'ProvidedPhone', 'RequestedVisit', 'NoCall', 'NoEmail']

In [1197]:
categorical_columns = ['CareLevel', 'Gender', 'Marital', 'CurrentResidence', 'UnitPreference', 'Budget', 'ReferralCompany', 'CurrentStatus']

In [1198]:
numerical_columns = ['AgeatInquiry']

In [1199]:
#nondata_columns = ['DeactReason', 'NotQual', 'ResidentID', 'dtmovein', 'dtmoveout', 'istatus', 'ireason']

In [1200]:
# CommunityCode, CommunityZipCode, ZipCode, InquiryDate, DeactReason, NotQual, ResidentID, dtmovein, dtmoveout, istatus, ireason
other_columns = list(set(list(ds13.columns)) - set(id_column) - set(target_column) - set(boolean_columns) - set(categorical_columns) - set(numerical_columns))

In [1201]:
# add column for Converted Flag, where Resident ID is populated
ds13['converted'] = np.where(ds13['ResidentID'].isnull(), 0, 1)

In [1202]:
ds13['converted'].value_counts()

0    172800
1      5738
Name: converted, dtype: int64

In [1203]:
ds13['AgeatInquiry'].min(axis=0)

-31.0

In [1204]:
ds13[ds13.AgeatInquiry == ds13['AgeatInquiry'].min(axis=0)]

,CommunityCode,CommunityZipCode,ProspectID,AgeatInquiry,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,...,DeactDate,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted
176671,863.0,85382.0,7eea047e-3989-4f40-b468-c8ba5a79c4ff,-31.0,A,Female,Divorced,False,NaN,98008.0,...,NaN,NaN,NaN,Moved In,NaN,NaN,NaN,NaN,NaN,0


In [1205]:
ds13[ds13.NotQual == 'Not Age Qualified']['AgeatInquiry'].min(axis=0)

-1.0

In [1206]:
ds13[ds13.AgeatInquiry == ds13[ds13.NotQual == 'Not Age Qualified']['AgeatInquiry'].min(axis=0)]

,CommunityCode,CommunityZipCode,ProspectID,AgeatInquiry,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,...,DeactDate,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted
12994,NaN,NaN,5c3e3671-c7ce-e611-80f6-5065f38a19e1,-1.0,F,Male,NaN,True,NaN,NaN,...,2017-03-23 04:00:00,Request Removal,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0
49261,692.0,45236.0,1bc86837-04f0-e711-80f0-e0071b6a71c1,-1.0,M,Female,Married,False,Living at Home,NaN,...,NaN,NaN,NaN,Prospect,NaN,NaN,NaN,NaN,NaN,0
71936,503.0,8527.0,9ee8a186-a2a6-e611-80ee-5065f38b31d1,-1.0,M,Female,NaN,False,NaN,NaN,...,NaN,NaN,NaN,Moved In,1049709.0,01/31/2017,03/04/2017,Moved Out,Higher Level Care,1
72031,426.0,33024.0,ffd164c5-85aa-e611-80ee-5065f38b31d1,-1.0,A,Male,NaN,True,Living at Home,NaN,...,NaN,NaN,NaN,Moved In,1050517.0,04/14/2017,NaN,Current,Unknown,1
72211,425.0,53158.0,691f5b9f-25be-e611-80f5-5065f38b31d1,-1.0,F,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,Moved In,1050187.0,02/11/2017,NaN,Current,NaN,1
72338,863.0,85382.0,8c4243f5-63cd-e611-80f9-5065f38b31d1,-1.0,F,Female,NaN,True,NaN,NaN,...,NaN,NaN,NaN,Moved In,1048806.0,01/15/2017,NaN,Current,NaN,1
82496,503.0,8527.0,fbcec593-e68e-e511-80e6-3863bb36cca0,-1.0,A,Female,NaN,False,Senior Community,NaN,...,NaN,NaN,NaN,Future,NaN,NaN,NaN,NaN,NaN,0
87052,863.0,85382.0,0c8980d5-edc2-e611-80f5-5065f38a19e1,-1.0,A,Female,NaN,True,NaN,NaN,...,NaN,NaN,NaN,Moved In,1049077.0,01/28/2017,NaN,Current,NaN,1
87229,491.0,30052.0,4a704218-c0ce-e611-80f6-5065f38a19e1,-1.0,A,Female,NaN,True,Living at Home,NaN,...,2017-01-24 05:00:00,Duplicate Lead,NaN,Future,NaN,NaN,NaN,NaN,NaN,0
94370,426.0,33024.0,2f822d43-4b28-e611-80fe-3863bb2e0220,-1.0,M,Female,NaN,True,Living at Home,NaN,...,NaN,NaN,NaN,Moved In,1050409.0,01/31/2017,NaN,Current,NaN,1


In [1207]:
# Clean-Up Age-At-Inquiry
#ds13[ds13['AgeatInquiry'] < 21]
#ds13[ds13['AgeatInquiry'] > 116]
ds13['age_cleaned'] = ds13['AgeatInquiry']
ds13['age_cleaned'] = np.where(ds13['age_cleaned'] < 21, np.nan, ds13['age_cleaned'])
ds13['age_cleaned'] = np.where(ds13['age_cleaned'] > 116, np.nan, ds13['age_cleaned'])
ds13['age_cleaned'].mean() # 81.760975225801374
#ds13['age_cleaned'].mean().round(decimals=0)
ds13['age_cleaned'] = ds13['age_cleaned'].fillna(ds13['age_cleaned'].mean())
ds13.head()

,CommunityCode,CommunityZipCode,ProspectID,AgeatInquiry,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
0,731.0,54311.0,0f0d9652-7697-4d03-84da-000137b878db,NaN,A,Female,NaN,False,NaN,NaN,...,Request Removal,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975
1,415.0,60010.0,59a04927-19ee-42de-840f-545a4e8b5533,77.0,I,Female,Widow,False,Living at Home,NaN,...,NaN,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,77.000000
2,491.0,30052.0,87305eb0-2ed2-430c-9b4e-00019a38134a,NaN,NaN,Female,NaN,False,NaN,NaN,...,Duplicate Lead,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975
3,775.0,33462.0,50dd6777-4333-48bb-9ac5-00041d493ef0,NaN,NaN,Female,NaN,False,NaN,NaN,...,Duplicate Lead,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975
4,492.0,33021.0,f7ce16ab-bbaf-45ae-be5b-0004e020269f,NaN,A,Female,NaN,False,NaN,NaN,...,Not Qualified,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975


In [1208]:
# Drop 'AgeatInquiry', since it has been replaced by 'age_cleaned'
ds13 = ds13.drop(['AgeatInquiry'], axis = 1)
ds13.head()

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
0,731.0,54311.0,0f0d9652-7697-4d03-84da-000137b878db,A,Female,NaN,False,NaN,NaN,NaN,...,Request Removal,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975
1,415.0,60010.0,59a04927-19ee-42de-840f-545a4e8b5533,I,Female,Widow,False,Living at Home,NaN,NaN,...,NaN,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,77.000000
2,491.0,30052.0,87305eb0-2ed2-430c-9b4e-00019a38134a,NaN,Female,NaN,False,NaN,NaN,NaN,...,Duplicate Lead,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975
3,775.0,33462.0,50dd6777-4333-48bb-9ac5-00041d493ef0,NaN,Female,NaN,False,NaN,NaN,NaN,...,Duplicate Lead,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975
4,492.0,33021.0,f7ce16ab-bbaf-45ae-be5b-0004e020269f,A,Female,NaN,False,NaN,NaN,NaN,...,Not Qualified,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975


In [1209]:
ds13['DeactDate'].min(axis=0)

inf

In [1210]:
ds13['DeactDate'] = ds13['DeactDate'].replace([np.inf, -np.inf], np.nan)

In [1211]:
ds13['DeactDate'].min(axis=0)

inf

In [1212]:
# WTF, Mate?
ds13[ds13.DeactDate == np.inf]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned


In [1213]:
ds13[ds13.DeactDate != np.nan]['DeactDate'].min(axis=0)

inf

In [1214]:
# For The Love Of God And All That Is Holy!!
ds13[ds13.DeactDate != np.inf]['DeactDate'].min(axis=0)

inf

In [1215]:
ds13['DeactDate'].max(axis=0)

'3015-04-01 05:00:00'

In [1216]:
ds13[ds13.DeactDate == "3015-04-01 05:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
177829,733.0,53228.0,2df06fc6-dd83-e411-a8dc-6c3be5a802b4,I,Female,NaN,False,NaN,NaN,NaN,...,Location,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.0


In [1217]:
# ds13[ds13.DeactDate == "3015-04-01 05:00:00"] = "2015-04-01 05:00:00"
ds13['DeactDate'] = np.where(ds13['DeactDate'] == "3015-04-01 05:00:00", "2015-04-01 05:00:00", ds13['DeactDate'])

In [1218]:
ds13[ds13.DeactDate == "3015-04-01 05:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned


In [1219]:
ds13['DeactDate'].max(axis=0)

'3014-02-16 06:00:00'

In [1220]:
ds13[ds13.DeactDate == "3014-02-16 06:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
97352,856.0,38119.0,b90e91a1-c8cc-44ba-9778-98b2f1107920,A,Female,NaN,False,NaN,NaN,NaN,...,Request Removal,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975


In [1221]:
# ds13[ds13.DeactDate == "3014-02-16 06:00:00"]['DeactDate'] = "2014-02-16 06:00:00"
#A value is trying to be set on a copy of a slice from a DataFrame.
ds13['DeactDate'] = np.where(ds13['DeactDate'] == "3014-02-16 06:00:00", "2014-02-16 06:00:00", ds13['DeactDate'])

In [1222]:
ds13[ds13.DeactDate == "3014-02-16 06:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned


In [1223]:
ds13['DeactDate'].max(axis=0)

'2105-02-24 05:00:00'

In [1224]:
ds13[ds13.DeactDate == "2105-02-24 05:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
5806,NaN,NaN,ebc5bfb9-e715-4af5-acbc-2a18bab7c6d8,A,Female,NaN,False,NaN,NaN,NaN,...,Request Removal,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,81.760975


In [1225]:
# ds13[ds13.DeactDate == "2105-02-24 05:00:00"] = "2015-02-24 05:00:00"
ds13['DeactDate'] = np.where(ds13['DeactDate'] == "2105-02-24 05:00:00", "2015-02-24 05:00:00", ds13['DeactDate'])

In [1226]:
ds13[ds13.DeactDate == "2105-02-24 05:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned


In [1227]:
ds13['DeactDate'].max(axis=0)

'2019-03-30 04:00:00'

In [1228]:
ds13[ds13.DeactDate == "2019-03-30 04:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
132474,498.0,33881.0,7d979dca-ddb5-e711-8111-5065f38af811,I,Male,NaN,True,NaN,33844.0,NaN,...,Moved In with Family,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,77.0


In [1229]:
# ds13[ds13.DeactDate == "2019-03-30 04:00:00"] = "2018-03-30 04:00:00"
ds13['DeactDate'] = np.where(ds13['DeactDate'] == "2019-03-30 04:00:00", "2018-03-30 04:00:00", ds13['DeactDate'])

In [1230]:
ds13[ds13.DeactDate == "2019-03-30 04:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned


In [1231]:
ds13['DeactDate'].max(axis=0)

'2018-11-28 06:00:00'

In [1232]:
ds13[ds13.DeactDate == "2018-11-28 06:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned
156307,684.0,61866.0,f2e02b82-01f2-e611-80fa-5065f38af811,F,Female,Widow,True,Living at Home,60918.0,NaN,...,Location,NaN,Deactivated,NaN,NaN,NaN,NaN,NaN,0,86.0


In [1233]:
# ds13[ds13.DeactDate == "2018-11-28 06:00:00"] = "2017-11-28 06:00:00"
ds13['DeactDate'] = np.where(ds13['DeactDate'] == "2018-11-28 06:00:00", "2017-11-28 06:00:00", ds13['DeactDate'])

In [1234]:
ds13[ds13.DeactDate == "2018-11-28 06:00:00"]

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,DeactReason,NotQual,CurrentStatus,ResidentID,dtmovein,dtmoveout,istatus,ireason,converted,age_cleaned


In [1235]:
# g2g
ds13['DeactDate'].max(axis=0)

'2018-04-27 07:00:00'

In [1236]:
# type "O" just stands for "object", which in Pandas’ world is a string (text).
ds13['InquiryDate'].dtype

dtype('O')

In [1237]:
ds13['ProjectedMoveIn'].dtype

dtype('O')

In [1238]:
ds13['DeactDate'].dtype

dtype('O')

In [1239]:
ds13['dtmovein'].dtype

dtype('O')

In [1240]:
ds13['dtmoveout'].dtype

dtype('O')

In [1241]:
ds13['InquiryDate'] = pd.to_datetime(ds13['InquiryDate'])
ds13['ProjectedMoveIn'] = pd.to_datetime(ds13['ProjectedMoveIn'])
ds13['DeactDate'] = pd.to_datetime(ds13['DeactDate'])
ds13['dtmovein'] = pd.to_datetime(ds13['dtmovein'])
ds13['dtmoveout'] = pd.to_datetime(ds13['dtmoveout'])

In [1242]:
# datetime64[ns] is a general dtype, while <M8[ns] is a specific dtype.
# On a computer whose byte order is little endian (*.x86), there is no difference between them.
ds13['InquiryDate'].dtype

dtype('<M8[ns]')

In [1243]:
ds13['ProjectedMoveIn'].dtype

dtype('<M8[ns]')

In [1244]:
ds13['DeactDate'].dtype

dtype('<M8[ns]')

In [1245]:
ds13['InquiryDate'].dtype

dtype('<M8[ns]')

In [1246]:
ds13['dtmovein'].dtype

dtype('<M8[ns]')

In [1247]:
ds13['dtmoveout'].dtype

dtype('<M8[ns]')

In [1248]:
ds13[categorical_columns] = ds13[categorical_columns].fillna(value = "MISSING")

In [1249]:
# create dummy variables for DataSet1
# CareLevel, Gender, Marital, CurrentResidence, UnitPreference, Budget, ReferralCompany
ds13 = pd.concat([ds13, pd.get_dummies(ds13['CareLevel'], prefix='CareLevel')], axis=1)
ds13 = pd.concat([ds13, pd.get_dummies(ds13['Gender'], prefix='Gender')], axis=1)
ds13 = pd.concat([ds13, pd.get_dummies(ds13['Marital'], prefix='Marital')], axis=1)
ds13 = pd.concat([ds13, pd.get_dummies(ds13['CurrentResidence'], prefix='CurrentResidence')], axis=1)
ds13 = pd.concat([ds13, pd.get_dummies(ds13['UnitPreference'], prefix='UnitPreference')], axis=1)
ds13 = pd.concat([ds13, pd.get_dummies(ds13['Budget'], prefix='Budget')], axis=1)
ds13 = pd.concat([ds13, pd.get_dummies(ds13['ReferralCompany'], prefix='ReferralCompany')], axis=1)
ds13.head()

,CommunityCode,CommunityZipCode,ProspectID,CareLevel,Gender,Marital,Veteran,CurrentResidence,ZipCode,UnitPreference,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
0,731.0,54311.0,0f0d9652-7697-4d03-84da-000137b878db,A,Female,MISSING,False,MISSING,NaN,MISSING,...,0,0,0,0,1,0,0,0,1,0
1,415.0,60010.0,59a04927-19ee-42de-840f-545a4e8b5533,I,Female,Widow,False,Living at Home,NaN,MISSING,...,0,0,0,0,1,1,0,0,0,0
2,491.0,30052.0,87305eb0-2ed2-430c-9b4e-00019a38134a,MISSING,Female,MISSING,False,MISSING,NaN,MISSING,...,0,0,0,0,1,1,0,0,0,0
3,775.0,33462.0,50dd6777-4333-48bb-9ac5-00041d493ef0,MISSING,Female,MISSING,False,MISSING,NaN,MISSING,...,0,0,0,0,1,0,1,0,0,0
4,492.0,33021.0,f7ce16ab-bbaf-45ae-be5b-0004e020269f,A,Female,MISSING,False,MISSING,NaN,MISSING,...,0,0,0,0,1,0,1,0,0,0


In [1250]:
# drop the original columns used to create the dummy variables
ds13 = ds13.drop(['CareLevel'], axis = 1)
ds13 = ds13.drop(['Gender'], axis = 1)
ds13 = ds13.drop(['Marital'], axis = 1)
ds13 = ds13.drop(['CurrentResidence'], axis = 1)
ds13 = ds13.drop(['UnitPreference'], axis = 1)
ds13 = ds13.drop(['Budget'], axis = 1)
ds13 = ds13.drop(['ReferralCompany'], axis = 1)
ds13.head()

,CommunityCode,CommunityZipCode,ProspectID,Veteran,ZipCode,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
0,731.0,54311.0,0f0d9652-7697-4d03-84da-000137b878db,False,NaN,False,False,False,False,False,...,0,0,0,0,1,0,0,0,1,0
1,415.0,60010.0,59a04927-19ee-42de-840f-545a4e8b5533,False,NaN,False,False,False,False,True,...,0,0,0,0,1,1,0,0,0,0
2,491.0,30052.0,87305eb0-2ed2-430c-9b4e-00019a38134a,False,NaN,False,False,False,False,True,...,0,0,0,0,1,1,0,0,0,0
3,775.0,33462.0,50dd6777-4333-48bb-9ac5-00041d493ef0,False,NaN,False,False,False,True,True,...,0,0,0,0,1,0,1,0,0,0
4,492.0,33021.0,f7ce16ab-bbaf-45ae-be5b-0004e020269f,False,NaN,False,False,False,False,True,...,0,0,0,0,1,0,1,0,0,0


In [1251]:
# For the boolean columns, replace TRUE and FALSE with 1 and 0
# Veteran, ProvidedEmail, ProvidedPhone, RequestedVisit, NoCall, NoEmail
ds13['Veteran'] = ds13['Veteran'].replace((True,False),(1,0))
ds13['ProvidedEmail'] = ds13['ProvidedEmail'].replace((True,False),(1,0))
ds13['ProvidedPhone'] = ds13['ProvidedPhone'].replace((True,False),(1,0))
ds13['RequestedVisit'] = ds13['RequestedVisit'].replace((True,False),(1,0))
ds13['NoCall'] = ds13['NoCall'].replace((True,False),(1,0))
ds13['NoEmail'] = ds13['NoEmail'].replace((True,False),(1,0))
ds13.head()

,CommunityCode,CommunityZipCode,ProspectID,Veteran,ZipCode,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
0,731.0,54311.0,0f0d9652-7697-4d03-84da-000137b878db,0,NaN,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,415.0,60010.0,59a04927-19ee-42de-840f-545a4e8b5533,0,NaN,0,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
2,491.0,30052.0,87305eb0-2ed2-430c-9b4e-00019a38134a,0,NaN,0,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
3,775.0,33462.0,50dd6777-4333-48bb-9ac5-00041d493ef0,0,NaN,0,0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
4,492.0,33021.0,f7ce16ab-bbaf-45ae-be5b-0004e020269f,0,NaN,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0


In [1252]:
ds13[other_columns] = ds13[other_columns].fillna(value = "MISSING")

In [1253]:
# Drop the rest of the non-data columns
# DataSet1: CommunityCode, CommunityZipCode, ProspectID, InquiryDate, ProjectedMoveIn, DeactDate, DeactReason, NotQual, CurrentStatus
# DataSet3: ResidentID, dtmovein, dtmoveout, istatus, ireason
ds13 = ds13.drop(['CommunityCode'], axis = 1)
ds13 = ds13.drop(['CommunityZipCode'], axis = 1)
ds13 = ds13.drop(['ProspectID'], axis = 1)
ds13 = ds13.drop(['InquiryDate'], axis = 1)
ds13 = ds13.drop(['ProjectedMoveIn'], axis = 1)
ds13 = ds13.drop(['DeactDate'], axis = 1)
ds13 = ds13.drop(['DeactReason'], axis = 1)
ds13 = ds13.drop(['NotQual'], axis = 1)
ds13 = ds13.drop(['CurrentStatus'], axis = 1)
ds13 = ds13.drop(['ResidentID'], axis = 1)
ds13 = ds13.drop(['dtmovein'], axis = 1)
ds13 = ds13.drop(['dtmoveout'], axis = 1)
ds13 = ds13.drop(['istatus'], axis = 1)
ds13 = ds13.drop(['ireason'], axis = 1)
ds13.columns.tolist()

['Veteran',
 'ZipCode',
 'ProvidedEmail',
 'ProvidedPhone',
 'RequestedVisit',
 'NoCall',
 'NoEmail',
 'converted',
 'age_cleaned',
 'CareLevel_A',
 'CareLevel_F',
 'CareLevel_I',
 'CareLevel_M',
 'CareLevel_MISSING',
 'CareLevel_S',
 'Gender_Female',
 'Gender_MISSING',
 'Gender_Male',
 'Marital_Divorced',
 'Marital_MISSING',
 'Marital_Married',
 'Marital_Single',
 'Marital_Widow',
 'CurrentResidence_Living at Home',
 'CurrentResidence_Living with Family',
 'CurrentResidence_MISSING',
 'CurrentResidence_Rental Apartment',
 'CurrentResidence_Senior Community',
 'UnitPreference_1 Bed',
 'UnitPreference_2 Bed',
 'UnitPreference_3 Bed',
 'UnitPreference_Cottage/Villa',
 'UnitPreference_MISSING',
 'UnitPreference_Semi-Private',
 'UnitPreference_Studio',
 'Budget_0-1,000',
 'Budget_1,000-2,000',
 'Budget_2,000-3,000',
 'Budget_3,000-4,000',
 'Budget_4,000-5,000',
 'Budget_5,000 +',
 'Budget_MISSING',
 'ReferralCompany_A Place for Mom',
 'ReferralCompany_Caring.com',
 'ReferralCompany_Other',

In [1254]:
ds13.dtypes

Veteran                                  int64
ZipCode                                 object
ProvidedEmail                            int64
ProvidedPhone                            int64
RequestedVisit                           int64
NoCall                                   int64
NoEmail                                  int64
converted                                int32
age_cleaned                            float64
CareLevel_A                              uint8
CareLevel_F                              uint8
CareLevel_I                              uint8
CareLevel_M                              uint8
CareLevel_MISSING                        uint8
CareLevel_S                              uint8
Gender_Female                            uint8
Gender_MISSING                           uint8
Gender_Male                              uint8
Marital_Divorced                         uint8
Marital_MISSING                          uint8
Marital_Married                          uint8
Marital_Singl

In [1255]:
ds13['ZipCode'] = ds13['ZipCode'].astype(str)

In [1256]:
ds13.dtypes

Veteran                                  int64
ZipCode                                 object
ProvidedEmail                            int64
ProvidedPhone                            int64
RequestedVisit                           int64
NoCall                                   int64
NoEmail                                  int64
converted                                int32
age_cleaned                            float64
CareLevel_A                              uint8
CareLevel_F                              uint8
CareLevel_I                              uint8
CareLevel_M                              uint8
CareLevel_MISSING                        uint8
CareLevel_S                              uint8
Gender_Female                            uint8
Gender_MISSING                           uint8
Gender_Male                              uint8
Marital_Divorced                         uint8
Marital_MISSING                          uint8
Marital_Married                          uint8
Marital_Singl

In [1257]:
ds13['ZipCode'] = ds13['ZipCode'].astype('U')

In [1258]:
ds13.dtypes

Veteran                                  int64
ZipCode                                 object
ProvidedEmail                            int64
ProvidedPhone                            int64
RequestedVisit                           int64
NoCall                                   int64
NoEmail                                  int64
converted                                int32
age_cleaned                            float64
CareLevel_A                              uint8
CareLevel_F                              uint8
CareLevel_I                              uint8
CareLevel_M                              uint8
CareLevel_MISSING                        uint8
CareLevel_S                              uint8
Gender_Female                            uint8
Gender_MISSING                           uint8
Gender_Male                              uint8
Marital_Divorced                         uint8
Marital_MISSING                          uint8
Marital_Married                          uint8
Marital_Singl

In [1259]:
ds13['ZipCode'] = ds13['ZipCode'].astype('|S') # which will by default set the length to the max len it encounters

In [1260]:
ds13.dtypes

Veteran                                  int64
ZipCode                                 object
ProvidedEmail                            int64
ProvidedPhone                            int64
RequestedVisit                           int64
NoCall                                   int64
NoEmail                                  int64
converted                                int32
age_cleaned                            float64
CareLevel_A                              uint8
CareLevel_F                              uint8
CareLevel_I                              uint8
CareLevel_M                              uint8
CareLevel_MISSING                        uint8
CareLevel_S                              uint8
Gender_Female                            uint8
Gender_MISSING                           uint8
Gender_Male                              uint8
Marital_Divorced                         uint8
Marital_MISSING                          uint8
Marital_Married                          uint8
Marital_Singl

In [1261]:
ds13 = ds13.drop(['ZipCode'], axis = 1)

In [1262]:
ds13.dtypes

Veteran                                  int64
ProvidedEmail                            int64
ProvidedPhone                            int64
RequestedVisit                           int64
NoCall                                   int64
NoEmail                                  int64
converted                                int32
age_cleaned                            float64
CareLevel_A                              uint8
CareLevel_F                              uint8
CareLevel_I                              uint8
CareLevel_M                              uint8
CareLevel_MISSING                        uint8
CareLevel_S                              uint8
Gender_Female                            uint8
Gender_MISSING                           uint8
Gender_Male                              uint8
Marital_Divorced                         uint8
Marital_MISSING                          uint8
Marital_Married                          uint8
Marital_Single                           uint8
Marital_Widow

In [1263]:
np.any(np.isnan(ds13))

False

In [1264]:
np.all(np.isfinite(ds13))

True

In [1265]:
np.isnan(ds13)

,Veteran,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,converted,age_cleaned,CareLevel_A,CareLevel_F,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [1266]:
np.where(np.isnan(ds13))

(array([], dtype=int64), array([], dtype=int64))

In [1267]:
# MODELS & BOTTLES!?!?!

In [1268]:
ds13_converted = ds13[ds13.converted == 1]

In [1269]:
ds13_converted['converted'].value_counts()

1    5738
Name: converted, dtype: int64

In [1270]:
ds13_unconverted = ds13[ds13.converted != 1]

In [1271]:
ds13_unconverted['converted'].value_counts()

0    172800
Name: converted, dtype: int64

In [1272]:
ds13_unconverted_sampled = ds13_unconverted.sample(5738)

In [1273]:
ds13_unconverted_sampled['converted'].value_counts()

0    5738
Name: converted, dtype: int64

In [1274]:
frames = [ds13_converted, ds13_unconverted_sampled]
ds13_rejoined = pd.concat(frames)

In [1275]:
ds13_rejoined['converted'].value_counts()

1    5738
0    5738
Name: converted, dtype: int64

In [1276]:
ds13_rejoined.head()

,Veteran,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,converted,age_cleaned,CareLevel_A,CareLevel_F,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
186,1,0,0,0,0,0,1,92.000000,0,0,...,0,0,1,0,0,1,0,0,0,0
259,1,0,0,1,0,0,1,91.000000,1,0,...,0,1,0,0,0,0,1,0,0,0
311,0,0,0,0,0,0,1,86.000000,0,0,...,1,0,0,0,0,1,0,0,0,0
415,0,0,0,0,1,1,1,81.760975,1,0,...,0,0,0,0,1,1,0,0,0,0
423,0,0,1,0,0,1,1,81.760975,0,0,...,0,0,1,0,0,1,0,0,0,0


In [1277]:
target = ds13_rejoined[['converted']]
ds13_sans_converted = ds13_rejoined.drop(['converted'], axis = 1)

In [1278]:
ds_train, ds_test, convert_train, convert_test = train_test_split(ds13_sans_converted, target, test_size=0.2)

In [1279]:
ds_train.head()

,Veteran,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,age_cleaned,CareLevel_A,CareLevel_F,CareLevel_I,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
165985,1,0,0,0,0,0,87.000000,0,1,0,...,0,0,1,0,0,1,0,0,0,0
128805,1,0,0,0,0,0,86.000000,0,0,0,...,0,0,0,1,0,1,0,0,0,0
13850,1,0,0,1,0,0,81.760975,1,0,0,...,0,1,0,0,0,0,1,0,0,0
126004,1,0,1,0,0,1,81.760975,0,0,1,...,1,0,0,0,0,1,0,0,0,0
21437,0,1,1,1,0,0,81.760975,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [1280]:
ds_test.head()

,Veteran,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,age_cleaned,CareLevel_A,CareLevel_F,CareLevel_I,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
59082,0,0,0,1,0,0,80.000000,1,0,0,...,0,1,0,0,0,0,1,0,0,0
138014,0,0,0,0,0,1,67.000000,0,0,0,...,0,0,0,0,1,1,0,0,0,0
163095,0,0,0,1,0,0,91.000000,1,0,0,...,0,1,0,0,0,0,1,0,0,0
7670,0,0,1,1,0,0,74.000000,0,0,1,...,0,0,0,0,1,0,0,0,1,0
15368,1,1,1,1,0,0,81.760975,0,0,1,...,0,0,1,0,0,0,1,0,0,0


In [1281]:
convert_train.head()

,converted
165985,1
128805,1
13850,0
126004,0
21437,1


In [1282]:
convert_test.head()

,converted
59082,1
138014,1
163095,1
7670,0
15368,0


In [1283]:
convert_train['converted'].value_counts()

1    4595
0    4585
Name: converted, dtype: int64

In [1284]:
convert_test['converted'].value_counts()

0    1153
1    1143
Name: converted, dtype: int64

In [1160]:
# min_max = preprocessing.MinMaxScaler().fit(ds_train)
# ds_train_norm = min_max.transform(ds_train)
# ds_test_norm = min_max.transform(ds_test)

In [1161]:
#https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
# ds_train = ds_train.reset_index()

In [1285]:
np.any(np.isnan(ds_train))

False

In [1286]:
np.all(np.isfinite(ds_train))

True

In [1287]:
np.any(np.isnan(convert_train))

False

In [1288]:
np.all(np.isfinite(convert_train))

True

In [1289]:
ds_train.head()

,Veteran,ProvidedEmail,ProvidedPhone,RequestedVisit,NoCall,NoEmail,age_cleaned,CareLevel_A,CareLevel_F,CareLevel_I,...,"Budget_2,000-3,000","Budget_3,000-4,000","Budget_4,000-5,000","Budget_5,000 +",Budget_MISSING,ReferralCompany_A Place for Mom,ReferralCompany_Caring.com,ReferralCompany_Other,ReferralCompany_SeniorLiving.net,ReferralCompany_Seniorhomes.com
165985,1,0,0,0,0,0,87.000000,0,1,0,...,0,0,1,0,0,1,0,0,0,0
128805,1,0,0,0,0,0,86.000000,0,0,0,...,0,0,0,1,0,1,0,0,0,0
13850,1,0,0,1,0,0,81.760975,1,0,0,...,0,1,0,0,0,0,1,0,0,0
126004,1,0,1,0,0,1,81.760975,0,0,1,...,1,0,0,0,0,1,0,0,0,0
21437,0,1,1,1,0,0,81.760975,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [1290]:
convert_train.head()

,converted
165985,1
128805,1
13850,0
126004,0
21437,1


In [1291]:
ds_train.shape

(9180, 45)

In [1292]:
convert_train.shape

(9180, 1)

In [1293]:
logisticRegr = LogisticRegression()
logisticRegr.fit(ds_train, convert_train.values.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [1171]:
#print("Score=%.3f" % clf.score(X, grades["Letter"]))

NameError: name 'clf' is not defined

In [ ]:
#print(logisticRegr.score(ds_test, convert_test))

In [1294]:
predictions = logisticRegr.predict(ds_test)

In [1296]:
print(logisticRegr.score(ds_test, convert_test))

0.727787456446


In [ ]:
#cm = confusion_matrix(predictions, convert_test)

In [ ]:
#print(pd.DataFrame(cm, columns=labels, index=labels))
#print(pd.DataFrame(cm)

In [1182]:
#logit = sm.Logit(data['admit'], data[train_cols])
training_columns = ds13_rejoined.columns.tolist()
training_columns

['Veteran',
 'ProvidedEmail',
 'ProvidedPhone',
 'RequestedVisit',
 'NoCall',
 'NoEmail',
 'converted',
 'age_cleaned',
 'CareLevel_A',
 'CareLevel_F',
 'CareLevel_I',
 'CareLevel_M',
 'CareLevel_MISSING',
 'CareLevel_S',
 'Gender_Female',
 'Gender_MISSING',
 'Gender_Male',
 'Marital_Divorced',
 'Marital_MISSING',
 'Marital_Married',
 'Marital_Single',
 'Marital_Widow',
 'CurrentResidence_Living at Home',
 'CurrentResidence_Living with Family',
 'CurrentResidence_MISSING',
 'CurrentResidence_Rental Apartment',
 'CurrentResidence_Senior Community',
 'UnitPreference_1 Bed',
 'UnitPreference_2 Bed',
 'UnitPreference_3 Bed',
 'UnitPreference_Cottage/Villa',
 'UnitPreference_MISSING',
 'UnitPreference_Semi-Private',
 'UnitPreference_Studio',
 'Budget_0-1,000',
 'Budget_1,000-2,000',
 'Budget_2,000-3,000',
 'Budget_3,000-4,000',
 'Budget_4,000-5,000',
 'Budget_5,000 +',
 'Budget_MISSING',
 'ReferralCompany_A Place for Mom',
 'ReferralCompany_Caring.com',
 'ReferralCompany_Other',
 'ReferralC

In [1183]:
training_columns.remove('converted')
training_columns

['Veteran',
 'ProvidedEmail',
 'ProvidedPhone',
 'RequestedVisit',
 'NoCall',
 'NoEmail',
 'age_cleaned',
 'CareLevel_A',
 'CareLevel_F',
 'CareLevel_I',
 'CareLevel_M',
 'CareLevel_MISSING',
 'CareLevel_S',
 'Gender_Female',
 'Gender_MISSING',
 'Gender_Male',
 'Marital_Divorced',
 'Marital_MISSING',
 'Marital_Married',
 'Marital_Single',
 'Marital_Widow',
 'CurrentResidence_Living at Home',
 'CurrentResidence_Living with Family',
 'CurrentResidence_MISSING',
 'CurrentResidence_Rental Apartment',
 'CurrentResidence_Senior Community',
 'UnitPreference_1 Bed',
 'UnitPreference_2 Bed',
 'UnitPreference_3 Bed',
 'UnitPreference_Cottage/Villa',
 'UnitPreference_MISSING',
 'UnitPreference_Semi-Private',
 'UnitPreference_Studio',
 'Budget_0-1,000',
 'Budget_1,000-2,000',
 'Budget_2,000-3,000',
 'Budget_3,000-4,000',
 'Budget_4,000-5,000',
 'Budget_5,000 +',
 'Budget_MISSING',
 'ReferralCompany_A Place for Mom',
 'ReferralCompany_Caring.com',
 'ReferralCompany_Other',
 'ReferralCompany_SeniorL

In [1184]:
logit = sm.Logit(ds13_rejoined['converted'], ds13_rejoined[training_columns])

In [1185]:
result = logit.fit(maxiter=999)

         Current function value: 0.548983
         Iterations: 999


LinAlgError: Singular matrix

In [1186]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:               178538
Model:                          Logit   Df Residuals:                   178499
Method:                           MLE   Df Model:                           38
Date:                Sun, 29 Apr 2018   Pseudo R-squ.:                   1.000
Time:                        21:13:43   Log-Likelihood:              -0.030247
converged:                      False   LL-Null:                       -25370.
                                        LLR p-value:                     0.000
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
ProvidedEmail                           1.2187     33.824      0.036      0.971     -65.075      67.512
ProvidedPhone                          -1.4468     31.48